# Generative Performance
## Fitting models to player 2 (Trustee)

In [27]:
import sys
sys.path.append('/Users/bryangonzalez/Documents/Dartmouth/Research/TheoryOfMind_Bx_experiment/Code/models/')
sys.path.append('/Users/bryangonzalez/Documents/Dartmouth/Research/TheoryOfMind_Bx_experiment/Code/')
import pandas as pd
import numpy as np
from scipy.optimize import least_squares
import p2_cost_funcs
import comp_models
from utils import compute_aic, compute_bic


dat = pd.read_csv('../data/bx_data.csv', index_col=0)
#dat = dat.rename(columns={'trustee_clust':'p2_strat'})
#dat.to_csv('../data/bx_data.csv', sep=',')
results = pd.DataFrame(columns=['p2', 'p3', 'p2_strat', 'model', 'param1', 'param2', 'sse', 'aic', 'bic', 'fit_method'])



game = dat.loc[dat.turker_id == 3]
niters = 10

fitIters = np.zeros([niters, 5])

## Fit Rescorla Wagner model to p2 to simulate RW agent performing task

In [29]:
#Fit
for i in range(niters):
    param0 = np.random.uniform()
    fitIters[i,0:2] = [param0, np.nan]
    result_lsq = least_squares(fun=p2_cost_funcs.rw_costfun, x0=param0, args=(game, ), bounds=(0,1))

    param1 = result_lsq.x
    cost = result_lsq.cost
    fitIters[i, 2:5] = [param1, np.nan, cost]
cost_selected = np.min(fitIters[:, 4])
alpha = fitIters[fitIters[:,4]== cost_selected, 2][0]
sse = cost_selected* 2
results = results.append(pd.DataFrame(
    [[game['trustee'].values[0], game['turker_id'].values[0], game['p2_strat'].values[0],
      'rw', alpha,np.nan, sse, compute_aic(len(game), sse, 1), compute_bic(len(game), sse, 1),
      'least_squares_'+str(niters)+'iters'
      ]],
    columns=results.columns
))

,p2,p3,p2_strat,model,param1,param2,sse,aic,bic,fit_method
0,81,3,GA,rw,0.35703,NaN,2012.598318,205.155444,207.198495,least_squares_10iters
